In [1]:
import torch
import torch.nn as nn
## to clarify the following question
## does batchnorm2d need weight, bias, running var, running mean for inference?

# model established

class BCResNets(nn.Module):
    def __init__(self):
        super().__init__()
        self.build_network()
        
    def build_network(self):
        self.cnn_head = nn.Sequential(
            nn.Conv2d(1, 5, kernel_size=3, bias=False), 
            nn.BatchNorm2d(5),
            nn.ReLU6(True),
        )

    def forward(self, x):
        x = self.cnn_head(x)
        return x

# model parameter initialized 
model=BCResNets()
model.train()
def custom_init(m):
    if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.normal_(m.bias, mean=0, std=0.01)

model.apply(custom_init)

# dummy data 
model.eval()
x=torch.randn(1,1,4,4)
print(x)
y=model(x)
print(y)

# parameters exported 
params = model.state_dict()
torch.save(params, "model_BN.prm", pickle_protocol = 5)

for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())


tensor([[[[ 1.2920, -1.0263,  0.5823,  0.8537],
          [ 1.3690,  0.1974,  1.5865, -0.0495],
          [-0.9763,  0.4018, -0.4414, -0.5954],
          [ 0.0130, -0.3720,  0.3734, -0.0434]]]])
tensor([[[[0.1521, 0.0942],
          [0.5929, 0.0000]],

         [[0.0948, 0.0926],
          [0.0000, 0.0000]],

         [[0.4132, 0.2185],
          [0.0000, 0.0000]],

         [[0.0000, 0.0000],
          [0.0000, 0.0000]],

         [[0.0915, 0.5942],
          [0.0000, 0.0494]]]], grad_fn=<HardtanhBackward0>)
cnn_head.0.weight 	 torch.Size([5, 1, 3, 3])
cnn_head.1.weight 	 torch.Size([5])
cnn_head.1.bias 	 torch.Size([5])
cnn_head.1.running_mean 	 torch.Size([5])
cnn_head.1.running_var 	 torch.Size([5])
cnn_head.1.num_batches_tracked 	 torch.Size([])


In [14]:
params = torch.load("model_BN.prm", map_location = "cpu")
print(params.keys())
print(params['cnn_head.1.running_var'])

odict_keys(['cnn_head.0.weight', 'cnn_head.1.weight', 'cnn_head.1.bias', 'cnn_head.1.running_mean', 'cnn_head.1.running_var', 'cnn_head.1.num_batches_tracked'])
tensor([1., 1., 1., 1., 1.])


In [8]:
print(model.state_dict()['cnn_head.1.running_mean'])

tensor([0., 0., 0., 0., 0.])
